In [1]:
import requests
import pandas as pd
from datetime import datetime

In [2]:
API_KEY = "6217a673bce249f5ab55e0b8455f8db1" 
url = "https://newsapi.org/v2/everything"
params = {
    'q': 'startup OR funding OR unicorn OR valuation OR series A',
    'language': 'en',
    'sortBy': 'publishedAt',
    'pageSize': 100,  # max per page
    'apiKey': API_KEY
}
response = requests.get(url, params=params)
data = response.json()

In [3]:
if response.status_code == 200:
    articles = data['articles']
    df = pd.DataFrame(articles)
    df = df[['source', 'title', 'description', 'publishedAt', 'url']]
    df['source'] = df['source'].apply(lambda x: x['name'])
    df['publishedAt'] = pd.to_datetime(df['publishedAt'])
else:
    print("Error:", data.get("message", "Request failed"))

In [4]:
df.columns = ['Source', 'Title', 'Description', 'Published_Date', 'URL']
df.head(5)

,Source,Title,Description,Published_Date,URL
0,9to5Mac,"Apple releases watchOS 26 beta 3, here’s what ...",Apple has released watchOS 26 beta 3 for devel...,2025-07-07 17:13:13+00:00,https://9to5mac.com/2025/07/07/apple-releases-...
1,GlobeNewswire,National Music Centre Launches Exhibition Hono...,"On July 16, National Music Centre will unveil ...",2025-07-07 17:13:00+00:00,https://www.globenewswire.com/news-release/202...
2,Dazed,Lana del Rey is our First Lady,We went to see the singer at London’s Wembley ...,2025-07-07 17:13:00+00:00,https://www.dazeddigital.com/music/article/681...
3,KicksOnFire.com,Puma Carina 2.0 Girlpower Wmns White / Silver ...,The Puma Carina 2.0 Girlpower WMNS in White an...,2025-07-07 17:13:00+00:00,https://www.kicksonfire.com/puma-carina-2-0-gi...
4,Hot Hardware,Porsche 911 Club Debuts As A Members-Only Coll...,Porsche just pulled the wraps off the 2026 Por...,2025-07-07 17:13:00+00:00,https://hothardware.com/news/porsche-911-club-...


In [5]:
df.to_csv("startup_news_data.csv", index=False)
print("Saved to startup_news_data.csv")

Saved to startup_news_data.csv


In [6]:
!pip install textblob
from textblob import TextBlob

   ---------------------------------------- 0.0/624.3 kB ? eta -:--:--
   ---------------------------------------- 624.3/624.3 kB 4.9 MB/s eta 0:00:00


In [7]:
# Combine Title and Description for analysis
df['Text'] = df['Title'].fillna('') + '. ' + df['Description'].fillna('')

# Apply sentiment polarity
df['Polarity'] = df['Text'].apply(lambda x: TextBlob(x).sentiment.polarity)

In [8]:
def label_sentiment(score):
    if score > 0.1:
        return "Promising"
    elif score < -0.1:
        return "At Risk"
    else:
        return "Neutral"

df['Sentiment_Label'] = df['Polarity'].apply(label_sentiment)
df[['Title', 'Polarity', 'Sentiment_Label']].head(10)

,Title,Polarity,Sentiment_Label
0,"Apple releases watchOS 26 beta 3, here’s what ...",0.000000,Neutral
1,National Music Centre Launches Exhibition Hono...,0.166667,Promising
2,Lana del Rey is our First Lady,0.625000,Promising
3,Puma Carina 2.0 Girlpower Wmns White / Silver ...,0.033333,Neutral
4,Porsche 911 Club Debuts As A Members-Only Coll...,0.000000,Neutral
5,Former Arsenal sporting director Edu joins Forest,0.034091,Neutral
6,Former rivals place heavy bet on Katie Taylor-...,-0.100000,Neutral
7,"Tom Sawyer Island, Liberty Square Riverboat & ...",-0.100000,At Risk
8,Nation's longest actively tenured athletic dir...,-0.016667,Neutral
9,shapez 2 is aiming for the 1.0 update in Novem...,0.254545,Promising


In [11]:
df.to_csv(r"E:\startup_insights_project\startup_sentiment_final.csv", index=False)

In [12]:
print("Sentiment analysis complete and file saved as 'startup_sentiment_final.csv'")

Sentiment analysis complete and file saved as 'startup_sentiment_final.csv'
